In [1]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('../chapter_7/GoogleNews-vectors-negative300.bin', binary=True)

unable to import 'smart_open.gcs', disabling that module


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = F.relu(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden
    
    def initHidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [3]:
def make_id_dic(fname):
    id_dic = {}
    
    with open(fname) as file:
        for i, line in enumerate(file, 1):
            line = line.rstrip('\n')
            id_dic[line] = i
            
    return id_dic

In [4]:
def words2id(sentence, id_dic):
    import re
    import snowballstemmer
    
    words_id = []
    
    # 文字種の統一
    sentence = sentence.lower()
    
    # 数字の置き換え
    sentence = re.sub(r'[0-9]+', '0', sentence)
    
    # '-'を' 'に変換
    sentence = sentence.replace('-', ' ')
    
    words = sentence.split()
    
    # ステミング処理
    stemmer = snowballstemmer.stemmer('english')
    words2 = [stemmer.stemWord(word) for word in words]
    words = words2
    
    for word in words:
        if word in id_dic.keys():
            words_id.append(id_dic[word])
        else:
            words_id.append(0)
            
    return words_id

In [5]:
def id2vec(words_id, id_dic):
    words_vec = []
    
    for id in words_id:
        word_list = [k for k, v in id_dic.items() if v == id]
        if len(word_list) > 0:
            try:
                words_vec.append(torch.from_numpy(model[word_list[0]]).view(1,-1))
            except KeyError:
                words_vec.append(torch.zeros(1, 300))
        else:
            words_vec.append(torch.zeros(1, 300))
    
    return torch.cat(words_vec)

In [6]:
def make_dataset(fname, id_dic):
    with open(fname) as file:
        lines = file.readlines()
        lines_vec = []
        labels = np.zeros([len(lines), 1])
        
        for i, line in enumerate(lines):
            
            line = line.rstrip('\n')
            category = line.strip('\t')[0]
            title = line.split('\t')[1]
            
            # lines_vecの処理
            words_id = words2id(title, id_dic)
            words_vec = id2vec(words_id, id_dic)
            lines_vec.append(words_vec)
            
            # labelsの処理
            if category == 'b':
                labels[i] = 0
            elif category == 't':
                labels[i] = 1
            elif category == 'e':
                labels[i] = 2
            elif category == 'm':
                labels[i] = 3
    labels = torch.from_numpy(labels).long()
                
    return lines_vec, labels

In [7]:
def train(rnn, criterion, optimizer, batch_size, words_vec_batch, label_batch):
    hidden = rnn.initHidden(batch_size)
    rnn.zero_grad()
    optimizer.zero_grad()
    hidden = hidden.detach()
    
    for i in range(len(words_vec_batch)):
        output, hidden = rnn(words_vec_batch[i], hidden)
    loss = criterion(output, label_batch)
    loss.backward(retain_graph=True)
    optimizer.step()

In [8]:
def get_loss_and_accyracy(rnn, criterion, lines_vec, labels):
    words_vec_list = []
    label_list = []
    
    for words_vec, label in zip(lines_vec, labels):
        words_vec_list.append(words_vec)
        label_list.append(label)
    words_vec_all, label_all = batch_process(words_vec_list, label_list)
    
    hidden = rnn.initHidden(len(lines_vec))
    
    for i in range(len(words_vec_all)):
        output, hidden = rnn(words_vec_all[i], hidden)
    loss = criterion(output, label_all)
    pre_labels = torch.max(output, 1)[1]
    accuracy = (pre_labels == label_all).sum().item() / len(labels)
    
    return loss, accuracy

In [9]:
def get_loss_and_accyracy(rnn, criterion, lines_vec, labels):
    running_loss = 0
    correct_count = 0
    
    for words_vec, label in zip(lines_vec, labels):
        words_vec_batch, label_batch = batch_process([words_vec], [label])
        hidden = rnn.initHidden(1)
        
        for i in range(len(words_vec_batch)):
            output, hidden = rnn(words_vec_batch[i], hidden)
        loss = criterion(output, label)
        running_loss += loss
        pre_label = torch.max(output, 1)[1]
        if pre_label == label:
            correct_count += 1
            
    return running_loss / len(labels), correct_count / len(labels)

In [10]:
def batch_process(words_vec_list, label_list):
    # words_vec_batchの処理
    len_max = 0
    for words_vec in words_vec_list:
        if words_vec.size()[0] > len_max:
            len_max = words_vec.size()[0]
    words_vec_batch = torch.zeros(len_max, len(words_vec_list), 300)
    for i in range(len(words_vec_list)):
        for j in range(len(words_vec_list[i])):
            words_vec_batch[len_max - len(words_vec_list[i]) + j, i, :] = words_vec_list[i][j]
        
    # label_batchの処理
    label_batch = torch.zeros(len(label_list)).long()
    for i, label in enumerate(label_list):
        label_batch[i] = label
        
    return words_vec_batch, label_batch

In [17]:
import numpy as np
import torch.optim as optim

rnn = RNN(300, 50, 4) # RNN(d_w, d_h, L)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(rnn.parameters(), lr=pow(10, -2)) # lr: 検証の結果、10エポック目で最も精度が高かった数値

id_dic = make_id_dic('../chapter_6/train.feature.txt')
lines_vec, labels = make_dataset('../chapter_6/train.txt', id_dic)
lines_vec_test, labels_test = make_dataset('../chapter_6/test.txt', id_dic)
batch_size = 8

for epoch in range(10):
    words_vec_list = []
    label_list = []
    
    for i, dataset in enumerate(zip(lines_vec, labels), 1):
        words_vec_list.append(dataset[0])
        label_list.append(dataset[1])
        
        if i % batch_size == 0 or i == len(lines_vec):
            words_vec_batch, label_batch = batch_process(words_vec_list, label_list)
            train(rnn, criterion, optimizer, len(words_vec_list), words_vec_batch, label_batch)
            words_vec_list = []
            label_list = []
            
    loss_train, accuracy_train = get_loss_and_accyracy(rnn, criterion, lines_vec, labels)
    loss_test, accuracy_test = get_loss_and_accyracy(rnn, criterion, lines_vec_test, labels_test)
    print('[{0}] 訓練データ上の損失: {1}\t訓練データ上の正解率: {2}\t評価データ上の損失: {3}\t評価データ上の正解率: {4}'.format(epoch + 1, loss_train, accuracy_train, loss_test, accuracy_test))

[1] 訓練データ上の損失: 1.0445014238357544	訓練データ上の正解率: 0.591743119266055	評価データ上の損失: 1.0557955503463745	評価データ上の正解率: 0.5899550224887556
[2] 訓練データ上の損失: 0.9616107940673828	訓練データ上の正解率: 0.6486612993821381	評価データ上の損失: 0.9791384935379028	評価データ上の正解率: 0.643928035982009
[3] 訓練データ上の損失: 0.7388445734977722	訓練データ上の正解率: 0.735349185545778	評価データ上の損失: 0.7333962321281433	評価データ上の正解率: 0.739880059970015
[4] 訓練データ上の損失: 0.5770920515060425	訓練データ上の正解率: 0.780939898895338	評価データ上の損失: 0.5921319723129272	評価データ上の正解率: 0.7683658170914542
[5] 訓練データ上の損失: 0.5535715222358704	訓練データ上の正解率: 0.7879610559820258	評価データ上の損失: 0.580390989780426	評価データ上の正解率: 0.7758620689655172
[6] 訓練データ上の損失: 0.5365055799484253	訓練データ上の正解率: 0.7932035199400861	評価データ上の損失: 0.5726305246353149	評価データ上の正解率: 0.7766116941529235
[7] 訓練データ上の損失: 0.5223622918128967	訓練データ上の正解率: 0.799850215315484	評価データ上の損失: 0.5666617751121521	評価データ上の正解率: 0.7781109445277361
[8] 訓練データ上の損失: 0.5100193023681641	訓練データ上の正解率: 0.8044373712787868	評価データ上の損失: 0.561617910861969	評価データ上の正解率: 0.7818590704647677
